In [1]:
import hnswlib
import numpy as np

In [54]:
dim = 5
num_elements = 10
num_nei = 5
# Generating sample data
data = np.float32(np.random.random((num_elements, dim)))
data_labels = np.arange(num_elements)

# Declaring index
p = hnswlib.Index(space = 'l2', dim = dim) # possible options are l2, cosine or ip

# Initing index - the maximum number of elements should be known beforehand
p.init_index(max_elements = num_elements, ef_construction = 200, M = 16)

# Element insertion (can be called several times):
p.add_items(data, data_labels)

# Controlling the recall by setting ef:
p.set_ef(50) # ef should always be > k

# Query dataset, k - number of closest elements (returns 2 numpy arrays)
labels, distances = p.knn_query(data, k = num_nei)

In [55]:
np.linalg.norm(data[3]-data[0])*np.linalg.norm(data[3]-data[0])

0.4962667

In [60]:
I = np.array(list(range(num_elements))*(num_nei-1))
J = labels[:,1:].flatten('F')
V = distances[:,1:].flatten('F')

In [50]:
print(labels)
print(distances)

[[0 2 9 5 7]
 [1 7 8 4 9]
 [2 0 9 7 5]
 [3 8 5 4 7]
 [4 5 9 8 6]
 [5 0 8 6 9]
 [6 5 9 4 8]
 [7 1 2 9 5]
 [8 3 5 7 4]
 [9 2 0 7 5]]
[[0.         0.32622173 0.35531518 0.41242993 0.9207057 ]
 [0.         0.48018348 0.8570967  0.9288626  1.0566152 ]
 [0.         0.32622173 0.33570072 0.5210747  0.6813143 ]
 [0.         0.18875808 0.57291615 0.9289004  1.1151049 ]
 [0.         0.6142057  0.6679617  0.7783899  0.8595362 ]
 [0.         0.41242993 0.44016412 0.51243216 0.5628238 ]
 [0.         0.51243216 0.84828025 0.8595362  0.88191456]
 [0.         0.48018348 0.5210747  0.5498486  0.586231  ]
 [0.         0.18875808 0.44016412 0.66557336 0.7783899 ]
 [0.         0.33570072 0.35531518 0.5498486  0.5628238 ]]


In [51]:
kng = np.zeros((num_elements, num_elements))
for i, label in enumerate(labels):
    for index, neighbor in enumerate(label[1:]):
        kng[i][neighbor] = distances[i][index+1]
kng[kng==0]=np.inf

In [52]:
kng

array([[       inf,        inf, 0.32622173,        inf,        inf,
        0.41242993,        inf, 0.92070568,        inf, 0.35531518],
       [       inf,        inf,        inf,        inf, 0.92886257,
               inf,        inf, 0.48018348, 0.85709667, 1.05661523],
       [0.32622173,        inf,        inf,        inf,        inf,
        0.68131429,        inf, 0.52107471,        inf, 0.33570072],
       [       inf,        inf,        inf,        inf, 0.92890042,
        0.57291615,        inf, 1.11510491, 0.18875808,        inf],
       [       inf,        inf,        inf,        inf,        inf,
        0.61420572, 0.85953617,        inf, 0.77838987, 0.66796172],
       [0.41242993,        inf,        inf,        inf,        inf,
               inf, 0.51243216,        inf, 0.44016412, 0.56282377],
       [       inf,        inf,        inf,        inf, 0.85953617,
        0.51243216,        inf,        inf, 0.88191456, 0.84828025],
       [       inf, 0.48018348, 0.5210747

In [64]:
from scipy import sparse

A = sparse.coo_matrix((V,(I,J)),shape=(num_elements,num_elements))

In [65]:
A

<10x10 sparse matrix of type '<class 'numpy.float32'>'
	with 40 stored elements in COOrdinate format>